In [27]:
%matplotlib inline

import pandas as pd
import numpy as np
import string
import matplotlib.pyplot as plt
import re
import emoji
from langdetect import detect
import unicodedata
from emoji import unicode_codes

In [12]:
data = pd.read_csv("../Data/cp_commments_sample.csv")
label = pd.read_csv("../Data/cp_comments_10k_labeled.csv")
label['comment_id'] = label['comment_id'].astype(int)

In [13]:
data_labeled = data.merge(label, left_on='comment_id', right_on='comment_id', how='right')
print(data_labeled.shape)
data_labeled = data_labeled[np.isfinite(data_labeled['media_id'])]
print(data_labeled.shape)

(10000, 11)
(4891, 11)


In [30]:
## Note that this emoji detector also throws out foriegn text
## e.g., 'Sabía'
#def is_emoji(text):
#    try:
#        text.encode('ascii')
#        return False
#    except UnicodeEncodeError:
#        return True

#myre = re.compile(u'['
#    u'\U0001F300-\U0001F64F'
#    u'\U0001F680-\U0001F6FF'
#    u'\u2600-\u26FF\u2700-\u27BF]+', 
#    re.UNICODE)

myre = re.compile(u'(%s[a-zA-Z0-9\+\-_&.ô’Åéãíç()!#*]+%s)')

#def char_is_emoji(character):
#    return character in emoji.UNICODE_EMOJI


def text_has_emoji(text):
    for character in text:
        if character in emoji.UNICODE_EMOJI:
        #if myre.search(character):
            return 1
    return 0

def text_has_partial_emoji(text):
    has_emoji = False
    has_text = False
    for character in text:
        if character in emoji.UNICODE_EMOJI:
        #if myre.search(character):
            has_emoji = True
        if character not in emoji.UNICODE_EMOJI:
        #if not myre.search(character):
            has_text = True
    return has_emoji & has_text
        
def get_text_from_partial_emoji(text):
    new_text = ''
    if(text_has_partial_emoji(text)):
        for character in text:
            if character not in emoji.UNICODE_EMOJI:
            #if not myre.search(character):
                new_text = new_text + character
    return new_text

def text_return_emoji(text):
    new_text = ''
    for character in text:
        if character in emoji.UNICODE_EMOJI:
        #if myre.search(character):
            new_text = new_text + character
    return new_text


def translate_emoji(text):
    new_text = ''
    for character in text:
        if character in emoji.UNICODE_EMOJI:
        #if myre.search(character):
            #new_text = new_text + re.sub(u'(%s[a-zA-Z0-9\+\-_&.ô’Åéãíç()!#*]+%s)', lambda m: unicodedata.name(m.group()), character) + '. '
            new_text = new_text + emoji.demojize(character, ('','')) + '. '
    return new_text

def get_emoji(word_list):
    new_text = ' '.join([text_return_emoji(word) for word in word_list.split()
                       if text_has_emoji(word)])
    return new_text
    

def is_username(word):
    if re.match("@[A-Za-z0-9_.]+",word):
        return True
    return False

def get_username(word_list):
    new_text = ' '.join([word.lower() for word in word_list.split()
                       if is_username(word)])
    return new_text
    
def is_hashtag(word):
    if re.match("#[A-Za-z0-9_.]+",word):
        return True
    return False

def get_hashtag(word_list):
    new_text = ' '.join([word.lower() for word in word_list.split()
                       if is_hashtag(word)])
    return new_text


def get_text(word_list):
    new_text = ''
    for word in word_list.split():
        if not is_username(word) and not is_hashtag(word) and not text_has_emoji(word):
            new_text = new_text + ' ' + word
        if text_has_partial_emoji(word):
            new_text = new_text + ' ' + get_text_from_partial_emoji(word)
    return new_text


def detect_language(text):
    if text is not '':
            lang = detect(text)
            return lang
    else:
        return None

In [15]:
parsed_comments = pd.DataFrame(columns = ['original_comment','text','language','emoji','translated_emoji','username','hashtag','label','purchase_int'])
parsed_comments['original_comment'] = data_labeled['comment_text']
parsed_comments['label'] = data_labeled['label']
parsed_comments['emoji'] = ''
parsed_comments['translated_emoji'] = ''

In [31]:
for index, row in parsed_comments.iterrows():
    parsed_comments.set_value(index,'username',get_username(row['original_comment']))
    parsed_comments.set_value(index,'hashtag',get_hashtag(row['original_comment']))
    parsed_comments.set_value(index,'emoji',get_emoji(row['original_comment']))
    parsed_comments.set_value(index,'text',get_text(row['original_comment']))

for index, row in parsed_comments.iterrows():    
    #parsed_comments.set_value(index,'language',detect_language(row['text']))
    parsed_comments.set_value(index,'translated_emoji',translate_emoji(row['emoji']))
    if parsed_comments['label'][index] == 4:
        parsed_comments.set_value(index,'purchase_int',int(1))
    else:
        parsed_comments.set_value(index,'purchase_int',int(0))

parsed_comments.to_csv('parsed_comments.csv', sep = ',')
parsed_comments

,original_comment,text,language,emoji,translated_emoji,username,hashtag,label,purchase_int
0,😍😍😍,,NaN,😍😍😍,smiling_face_with_heart-eyes. smiling_face_wit...,,,1,0
1,Love the make up @semaashraf,Love the make up,NaN,,,@semaashraf,,4,1
2,👌👌👌beauuutifulll makeup!,beauuutifulll makeup!,NaN,👌👌👌,OK_hand. OK_hand. OK_hand.,,,1,0
3,This campaign 😍💗,This campaign,NaN,😍💗,smiling_face_with_heart-eyes. growing_heart.,,,1,0
4,New summer gloss in amber though😩😍😍,New summer gloss in amber though,NaN,😩😍😍,weary_face. smiling_face_with_heart-eyes. smil...,,,1,0
5,❤️the brows @japanese.latina,️the brows,NaN,❤,red_heart.,@japanese.latina,,1,0
6,💞💞,,NaN,💞💞,revolving_hearts. revolving_hearts.,,,1,0
7,Wow😍,Wow,NaN,😍,smiling_face_with_heart-eyes.,,,1,0
8,😍😊😊,,NaN,😍😊😊,smiling_face_with_heart-eyes. smiling_face_wit...,,,1,0
9,✔️,️,NaN,✔,heavy_check_mark.,,,1,0


In [22]:
help (emoji)

Help on package emoji:

NAME
    emoji

DESCRIPTION
    emoji for Python
    ~~~~~~~~~~~~~~~~
    
    emoji terminal output for Python.
    
        >>> import emoji
        >>> print(emoji.emojize('Python is :thumbsup:', use_aliases=True))
        Python is 👍
        >> print(emoji.emojize('Python is :thumbs_up_sign:'))
        Python is 👍

PACKAGE CONTENTS
    core
    unicode_codes

DATA
    EMOJI_ALIAS_UNICODE = {':+1:': '👍', ':100:': '💯', ':1234:': '🔢', ':1st...
    EMOJI_UNICODE = {':1st_place_medal:': '🥇', ':2nd_place_medal:': '🥈', '...
    UNICODE_EMOJI = {'# ️ ⃣': ':keycap_#:', '* ️ ⃣': ':keycap_*:', '0 ️ ⃣'...
    UNICODE_EMOJI_ALIAS = {'# ⃣': ':keycap_number_sign:', '# ️ ⃣': ':keyca...
    __email__ = 'carpedm20@gmail.com and wursterk@gmail.com'
    __license__ = '\nNew BSD License\n\nCopyright (c) 2014-2015, Taeho...E...
    __source__ = 'https://github.com/carpedm20/emoji/'

VERSION
    0.4.5

AUTHOR
    Taehoon Kim and Kevin Wurster

FILE
    /anaconda/lib/python3.6/site